# Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
import sklearn

# Read dataset

In [2]:
df = pd.read_csv('Combined_Flights_2018.csv', usecols=['Origin', 'Dest', 'Cancelled', 'CRSDepTime', 'DepDelay', 'CRSElapsedTime', 'Distance', 'Month', 'DayofMonth', 'DayOfWeek', 'IATA_Code_Operating_Airline', 'CRSArrTime', 'ArrDelay', 'ArrivalDelayGroups'])
print(df.head())
print(df.shape)

  Origin Dest  Cancelled  CRSDepTime  DepDelay  CRSElapsedTime  Distance  \
0    ABY  ATL      False        1202      -5.0            62.0     145.0   
1    ABY  ATL      False        1202      -5.0            62.0     145.0   
2    ABY  ATL      False        1202      -9.0            62.0     145.0   
3    ABY  ATL      False        1202     -12.0            62.0     145.0   
4    ABY  ATL      False        1400      -5.0            60.0     145.0   

   Month  DayofMonth  DayOfWeek IATA_Code_Operating_Airline  CRSArrTime  \
0      1          23          2                          9E        1304   
1      1          24          3                          9E        1304   
2      1          25          4                          9E        1304   
3      1          26          5                          9E        1304   
4      1          27          6                          9E        1500   

   ArrDelay  ArrivalDelayGroups  
0      -8.0                -1.0  
1      -6.0             

# Data preprocessing

## drop cancelled flights

In [3]:
df_col = df.loc[df['Cancelled'] == False]
print(df_col.shape)
df_col = df_col.drop('Cancelled', axis=1)
print(df_col.shape)
# simplfy column_names
df_col.rename(columns = {'CRSDepTime': 'DepTime', 'CRSElapsedTime': 'ElapsedTime','DayOfWeek': 'DayofWeek', 'IATA_Code_Operating_Airline': 'Airline', 'CRSArrTime': 'ArrTime', 'ArrivalDelayGroups':'ArrGroup'}, inplace = True)
print(df_col.head())

(5601139, 14)
(5601139, 13)
  Origin Dest  DepTime  DepDelay  ElapsedTime  Distance  Month  DayofMonth  \
0    ABY  ATL     1202      -5.0         62.0     145.0      1          23   
1    ABY  ATL     1202      -5.0         62.0     145.0      1          24   
2    ABY  ATL     1202      -9.0         62.0     145.0      1          25   
3    ABY  ATL     1202     -12.0         62.0     145.0      1          26   
4    ABY  ATL     1400      -5.0         60.0     145.0      1          27   

   DayofWeek Airline  ArrTime  ArrDelay  ArrGroup  
0          2      9E     1304      -8.0      -1.0  
1          3      9E     1304      -6.0      -1.0  
2          4      9E     1304      -2.0      -1.0  
3          5      9E     1304     -11.0      -1.0  
4          6      9E     1500      -1.0      -1.0  


## one-hot encoded

In [4]:
origin = pd.get_dummies(df_col['Origin'], prefix='origin')
print(origin.shape)
dest = pd.get_dummies(df_col['Dest'], prefix='dest')
print(dest.shape)
month = pd.get_dummies(df_col['Month'], prefix='month')
print(month.shape)
day_m = pd.get_dummies(df_col['DayofMonth'], prefix='day_m')
print(day_m.shape)
day_w = pd.get_dummies(df_col['DayofWeek'], prefix='day_w')
print(day_w.shape)
airline = pd.get_dummies(df_col['Airline'], prefix='airline')
print(airline.shape)

(5601139, 370)
(5601139, 370)
(5601139, 12)
(5601139, 31)
(5601139, 7)
(5601139, 28)


In [5]:
# drop original columns
df_col = df_col.drop(columns=['Origin', 'Dest', 'Month', 'DayofMonth', 'DayofWeek', 'Airline'])
print(df_col.shape)

# combine dataset together
df_new = pd.concat([df_col, origin], axis=1)
df_new = pd.concat([df_new, dest], axis=1)
df_new = pd.concat([df_new, month], axis=1)
df_new = pd.concat([df_new, day_m], axis=1)
df_new = pd.concat([df_new, day_w], axis=1)
df_new = pd.concat([df_new, airline], axis=1)
print(df_new.shape)

(5601139, 7)
(5601139, 825)


## Numerical data 

In [20]:
# Convert 24h-formatted time to minutes 
def time_to_minutes(time_24h):
    """Convert 24h-formatted time to minutes."""
    return (time_24h // 100) * 60 + (time_24h % 100)

df_new['DepTime'] = df_new['DepTime'].apply(time_to_minutes)
df_new['ArrTime'] = df_new['ArrTime'].apply(time_to_minutes)

##### Notes #####
''' 
ArrGroup is just the sign of ArrDelay (minutes) 
Distance is the distance between airports (miles) 
DepTime is the CRS departure time (HHMM) 

DepTime + DepDelay is the actual departure time (HHMM) 
We can normalize each column individually despite the relation between DepTime and DepDelay, 
since DepTime is going to tell us how busy the airport is at that time of day, and DepDelay is related to the flight itself. 
In other words, we treat them as separate features. 
''' 

# Normalize and standardize columns 
columns_to_normalize = ['DepTime', 'ArrTime', 'DepDelay', 'ArrDelay', 'ElapsedTime', 'Distance']

# Necessary imports 
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Initialize scalers
minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

##### Notes #####
'''
The rescaling of data depends on the model we are using. 
Here I apply both MinMaxScaler and StandardScaler to the data, and name the new columns accordingly.
The original columns are dropped at the end. 
'''

for column in columns_to_normalize:
    # Create normalized columns
    df_new[column + '_normalized'] = minmax_scaler.fit_transform(df_new[[column]])
    
    # Create standardized columns
    df_new[column + '_standardized'] = standard_scaler.fit_transform(df_new[[column]])
    
    # Drop the original columns
    df_new.drop(column, axis=1, inplace=True)


print(df_new[[columns_to_normalize].head()].head())

## Split dataset

In [6]:
from sklearn.model_selection import train_test_split
# for regression
X = df_new.drop(columns=['ArrDelay',  'ArrGroup'])
y = df_new['ArrDelay']
## for classification
# Y = df_new['ArrGroup']

# split train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
# split train - train/validate
X_training, X_val, y_training, y_val = train_test_split(X_train, y_train, test_size=0.11, random_state=42)

print(X_training.shape, X_val.shape, X_test.shape)
print(y_training.shape, y_val.shape, y_test.shape)

(4486512, 823) (554513, 823) (560114, 823)
(4486512,) (554513,) (560114,)
